In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
conf = pyspark.SparkConf().setAppName("Master").setMaster("local")
sc = SparkSession.builder.config(conf=conf).getOrCreate()

In [32]:
from pyspark.sql.functions import *
from pyspark.sql import Window
from pyspark.sql.types import *

In [5]:
def read_csv(path):
    return sc.read.csv(path, inferSchema=True, header=True)

In [ ]:
currency_df = read_csv("assets/currency.csv")
organizations_df = read_csv("assets/organizations.csv")

currency_df.show()

In [ ]:

organizations_df.show()  

In [ ]:
organizations_df.filter(organizations_df['Number of employees'] > 1000).count()

In [ ]:
organizations_df.groupBy('Country').count().filter('count > 4').sort('count', ascending=False).show()

In [ ]:
organizations_df.select(substring(organizations_df['Country'], 0, 4)).alias("First Three Letters").limit(5).show()

In [ ]:
organizations_df.filter(organizations_df['Country'] == 'Tunisia').select(instr(organizations_df.Country, 'i')).distinct().show()

In [ ]:
odf = organizations_df
odf.select(regexp_replace('Country', 'i', 'I')).show()

In [ ]:
odf.select(concat('Country', lit(' '), 'Founded')).show(truncate=False)

In [ ]:
odf.orderBy('Number of employees', ascending=False).orderBy(col('Country').asc()).show()

In [ ]:
# Query to fetch unique values of MAJOR Subjects from Student table.
# Query to print the first 3 characters of FIRST_NAME from Student table.
# Query to find the position of alphabet (‘a’) int the first name column ‘Shivansh’ from Student table.
# Query that fetches the unique values of MAJOR Subjects from Student table and print its length.
# Query to print FIRST_NAME from the Student table after replacing ‘a’ with ‘A’.
# Query to print the FIRST_NAME and LAST_NAME from Student table into single column COMPLETE_NAME.
# Query to print all Student details order by FIRST_NAME Ascending and MAJOR Subject descending .
# Query to print details of the Students excluding FIRST_NAME as ‘Prem’ and ‘Shivansh’.
odf.filter('COUNTRY NOT IN ("Tunisia", "Algeria")').show()
# Query to print details of the Students whose FIRST_NAME ends with ‘a’ and contains five alphabets.
odf.filter('COUNTRY LIKE "____a"').show()
# Query to fetch the no. of Students for each MAJOR subject in the descending order.
odf.groupBy('COUNTRY').count().orderBy("COUNT", ascending = False).show()
# Query to show only odd rows from Student table.
odf.select('*', row_number().over(Window.orderBy("Name")).alias("rn")).filter("rn %2 != 0").show()
# Query to fetch the list of Students with the same GPA.
odf.groupBy("COUNTRY").agg(collect_list("Name").alias("Names")).show(truncate=False)

New Section

In [ ]:
schema = StructType([
    StructField("name", IntegerType()),
    StructField("date", StringType()),
    StructField("filterrules", StringType()),
])

data = [
    (1, '2022-01-01', 'entire home,private room'),
    (2, '2022-01-02', 'entire home,shared room'),
    (3, '2022-01-02', 'private room,shared room'),
    (4, '2022-01-03', 'private room')
]

df = sc.createDataFrame(data=data, schema=schema)
df.show(truncate=False)

In [ ]:
df.select('name', 'date', explode(split("filterrules", ","))).show()

In [ ]:
columns = ['emp_id', 'designation']
# Create DataFrame for 2020 and
emp_2020_data = [(1, 'Trainee'), (2, 'Developer'), (3, 'Senior Developer'), (4, 'Manager')]
emp_2021_data = [(1, 'Developer'), (2, 'Developer'), (3, 'Manager'), (5, 'Trainee')]

emp_2020_df = sc.createDataFrame(emp_2020_data, columns)
emp_2021_df = sc.createDataFrame(emp_2021_data, columns)

emp_2020_df.show()
emp_2021_df.show()

In [ ]:
# Show status on each employee from year 2020 to 2021 if changed
# If new - NEW; If resigned - RESIGNED; If designation changed - PROMOTED
emp_2020_df.join(emp_2021_df, on="emp_id", how="fullouter").select(coalesce(emp_2021_df["emp_id"], emp_2020_df["emp_id"]).alias("emp_id"), when(emp_2020_df["designation"].isNull(), lit("NEW")).when(emp_2021_df["designation"].isNull(), lit("RESIGNED")).when(emp_2021_df["designation"] == emp_2020_df["designation"], emp_2020_df["designation"]).otherwise("PROMOTED").alias("STATUS")).show()